In [ ]:
import os
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
import numpy as np
path = '/home/adarsh-binny/HULK/Saved_models/'

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("../HULK/Counterspeech/models/createdebate_model",cache_dir=path)
model = AutoModelWithLMHead.from_pretrained("../HULK/Counterspeech/models/createdebate_model",cache_dir=path)

In [ ]:
model.eval()

In [ ]:
conditioned_tokens = []
generated_tokens = []
past = None
fst = True
DELIMITER = "/n"

def reinput(user_msg):
    global conditioned_tokens; global fst
    os.system('cls' if os.name == 'nt' else 'clear')

    conditioned_tokens = tokenizer.decode(conditioned_tokens)

    print("Me: " + user_msg + "\n" + "Bot: ",end='')

    if fst:
      fst = False
      user_msg = "<|endoftext|>" + user_msg

    user_msg = "" + user_msg
    conditioned_tokens += user_msg 
    conditioned_tokens = tokenizer.encode(conditioned_tokens) 
    conditioned_tokens += [50256] # Append operator to prepend conversation history



def top_p_filtering(logits, top_p=0.9, filter_value=-float('Inf')):
  """
  Credit: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
  """
  assert logits.dim() == 1  # batch size 1 for single word generation
  sorted_logits, sorted_indices = torch.sort(logits, descending=True)
  cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
  # remove tokens with cumulative probability above the threshold
  sorted_indices_to_remove = cumulative_probs > top_p
  # shift the indices to the right to keep also the first token above the threshold
  sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
  sorted_indices_to_remove[..., 0] = 0
  indices_to_remove = sorted_indices[sorted_indices_to_remove]
  logits[indices_to_remove] = filter_value
  return logits


def recalc(p=None):
    global conditioned_tokens
    global generated_tokens
    global past

    # for segment display purpose, keep 2 sets of tokens
    indexed_tokens = conditioned_tokens + generated_tokens

    tokens_tensor = torch.tensor([indexed_tokens])

    #tokens_tensor = tokens_tensor
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]
    logits = predictions[0, -1, :]
    filtered_logits = top_p_filtering(logits)
    probabilities = F.softmax(filtered_logits, dim=-1)
    next_token = torch.multinomial(probabilities, 1)
    generated_tokens.append(next_token.item())
    return next_token.item()

def generate():
    global conditioned_tokens
    global generated_tokens
    global past

    while True:
        if len(tokenizer.decode(conditioned_tokens)) > 320:
          dc = tokenizer.decode(conditioned_tokens)
          dc = dc[len(dc)-320:]
          idx = dc.find(DELIMITER)
          if idx != -1:
            dc = dc[idx+len(DELIMITER):]
            conditioned_tokens = tokenizer.encode(dc)

        result = recalc()

        if result == 50256:

      # end-of-text : 50256
      # use this special token to split segments

            decoded_reply = tokenizer.decode(generated_tokens)

            to_print = decoded_reply
            if to_print.endswith("<|endoftext|>"):
              to_print = to_print[:-len("<|endoftext|>")]
              decoded_reply = to_print
            print(to_print)

            decoded_reply = "" + decoded_reply
            decoded_reply = decoded_reply+DELIMITER
            conditioned_tokens += (tokenizer.encode(decoded_reply))

            # Remove end of text tokens from feeding back into model
      # -- see here for reason https://github.com/huggingface/transformers/issues/429#issuecomment-479380117
            cond_str = tokenizer.decode(conditioned_tokens)
            cond_str = cond_str.replace("<|endoftext|>",DELIMITER) # Since "<end-of-text> (50256)" is mapped to the newline character (\n) by default when calling decode():
            conditioned_tokens = tokenizer.encode(cond_str)

    # Uncomment to debug (print) conversation history that is re-fed to the model
    # cond_str = tokenizer.decode(conditioned_tokens)
    # cond_str = cond_str.replace("<|endoftext_|>","\n").replace(DELIMITER,"\n") # Since "<end-of-text> (50256)" is mapped to the newline character (\n) by default when calling decode():
    # print("(condstr_dbg) " + cond_str)
    # print(r"(/constr_dbg)")

            generated_tokens = []
            break

In [ ]:
from IPython.display import clear_output
import logging

from IPython.display import clear_output
import logging

def new_dialogue():

  global conditioned_tokens
  global generated_tokens
  global past; global fst
  
  conditioned_tokens = []; generated_tokens = []; past = None; fst = True
  reinput("What is the meaning of life?")
  generate()

  while True:
    cmd = input()

    if cmd == "reset":
      clear_output()
      new_dialogue()
      break

    if cmd != "":
      reinput(cmd)
    generate()
    

In [ ]:
new_dialogue()

In [2]:
import neptune.new as neptune
from apiconfig import project_name,api_token

run = neptune.init(project=project_name,api_token=api_token)

# Track metadata and hyperparameters of your run
run["JIRA"] = "NPT-952"
run["parameters"] = {"learning_rate": 0.001,
                     "optimizer": "Adam"}

# Track the training process by logging your training metrics
for epoch in range(100):
   run["train/accuracy"].log(epoch * 0.6)
   run["train/loss"].log(epoch * 0.4)

run["f1_score"] = 0.66
run.stop()

https://app.neptune.ai/Hatespeech-CNERG/Counterspeech-generation/e/COUNTER-2


In [ ]:
!pip install neptune-client==0.9

In [22]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

cache_path='../HULK/Saved_models/'

tokenizer = GPT2Tokenizer.from_pretrained('gpt2',cache_dir=cache_path)
model = GPT2LMHeadModel.from_pretrained('gpt2', return_dict=True,cache_dir=cache_path)

# when generating, we will use the logits of right-most token to predict the next token
# so the padding should be on the left
tokenizer.padding_side = "left" 
tokenizer.pad_token = tokenizer.eos_token # to avoid an error


sentences = ["Hello, my dog is a little",
             "Hello, my dog is", # use different length sentences to test batching
             "Hello there "
            ]
inputs = tokenizer.encode_plus(sentences[0],add_special_tokens=True,max_length=10, return_tensors="pt", padding=True)


output_sequences = model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    do_sample=False, # disable sampling to test if batching affects output
)

for i in range(len(sentences)):
    print(tokenizer.decode(output_sequences[i],skip_special_tokens=True)[len(sentences[i]):])
    print('======')
    # you can use skip_special_tokens=True in decode() to remove padding token
    # but note that it will also remove other special_tokens



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 bit of a mess. I'm not sure if he's going


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [23]:
inputs['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1]])

In [16]:
inputs

{'input_ids': tensor([[50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1]])}

In [29]:
inputs = tokenizer(sentences,max_length=10, padding=True)


In [30]:
inputs

{'input_ids': [[15496, 11, 616, 3290, 318, 257, 1310], [50256, 50256, 15496, 11, 616, 3290, 318], [50256, 50256, 50256, 50256, 15496, 612, 220]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1]]}

In [26]:
tokenizer.all_special_tokens

['<|endoftext|>', '<|endoftext|>']